In [5]:
%matplotlib notebook

import matplotlib.pyplot as plt
from IPython.display import display
import neurokit2 as nk
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.signal
import os
import pickle

# Set matplotlib parameters for displaying graphs
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = [10, 5]  # Bigger images
plt.rcParams['font.size']= 14

pd.set_option('display.max_columns', None)

In [3]:
cwd = os.getcwd()
home_dir = os.path.dirname(cwd)

In [4]:
filepaths = []
for subdir, dirs, files in os.walk(home_dir):
    for filename in files:
        filepath = subdir + os.sep + filename
        
        if filepath.endswith(".acq"):
            filepaths.append(filepath)
filepaths

['C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part201\\part201_baseline.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part201\\part201_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part202\\part202_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part206\\part206_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part207\\part207_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part208\\part208_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part209\\part209_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part210\\part210_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part211\\part211_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part212\\part212_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part213\\part2

In [8]:



# Read in data directly from .acq file and rename dataframe columns
# ACQ file has pre-named colums so you need to make them easier to deal with later in the script
# subject numbers are 201-215
# raw data is barely useful, in units of voltage (RSP), microsiemens (EDA), mV (ECG), and V (TRIG)

data, sampling_rate = nk.read_acqknowledge(filepaths[1])
data = data.rename(columns={"RSP, X, RSPEC-R": "RSP", "DTU100 - Trigger View, AMI / HLT - A11": "TRIG",
                            "EDA, X, PPGED-R": "EDA", "ECG, X, RSPEC-R": "ECG"})
print(data.head())

        RSP       EDA       ECG      TRIG
0  6.958313 -0.534233 -0.032654  8.494263
1  6.957703 -0.535759 -0.032043  8.424683
2  6.957703 -0.538811 -0.031281  8.443604
3  6.957703 -0.534233 -0.031128  8.554993
4  6.955261 -0.535759 -0.031128  8.601990


In [9]:
# Process ECG, EDA, and RSP over the entire experiment
# takes the unidimensional psychophys data and derives relevant features for each subject

ecg_signals, info1 = nk.ecg_process(data["ECG"], sampling_rate=2000)
eda_signals, info2 = nk.eda_process(data["EDA"], sampling_rate=2000)
rsp_signals, info3 = nk.rsp_process(data["RSP"], sampling_rate=2000)

In [10]:
ecg_signals

,ECG_Raw,ECG_Clean,ECG_Rate,ECG_Quality,ECG_R_Peaks,ECG_P_Peaks,ECG_Q_Peaks,ECG_S_Peaks,ECG_T_Peaks,ECG_P_Onsets,ECG_T_Offsets,ECG_Phase_Atrial,ECG_Phase_Completion_Atrial,ECG_Phase_Ventricular,ECG_Phase_Completion_Ventricular
0,-0.032654,-0.014469,78.388145,0.99836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
1,-0.032043,-0.014262,78.388145,0.99836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
2,-0.031281,-0.014054,78.388145,0.99836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
3,-0.031128,-0.013848,78.388145,0.99836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
4,-0.031128,-0.013642,78.388145,0.99836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7291571,0.002289,0.001692,74.906367,0.97120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
7291572,0.002441,0.001655,74.906367,0.97120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
7291573,0.002289,0.001615,74.906367,0.97120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
7291574,0.001526,0.001573,74.906367,0.97120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0


In [23]:
pickle.dump( ecg_signals, open( filepaths[1][:-10]+"_ecg_signals.p", "wb" ) )

In [24]:
diditwork = pickle.load( open( filepaths[1][:-10]+"_ecg_signals.p", "rb" ) )

In [25]:
display(nk.ecg_analyze(diditwork))

,ECG_Rate_Mean,HRV_RMSSD,HRV_MeanNN,HRV_SDNN,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_pNN50,HRV_pNN20,HRV_TINN,HRV_HTI,HRV_ULF,HRV_VLF,HRV_LF,HRV_HF,HRV_VHF,HRV_LFHF,HRV_LFn,HRV_HFn,HRV_LnHF,HRV_SD1,HRV_SD2,HRV_SD1SD2,HRV_S,HRV_CSI,HRV_CVI,HRV_CSI_Modified,HRV_PIP,HRV_IALS,HRV_PSS,HRV_PAS,HRV_GI,HRV_SI,HRV_AI,HRV_PI,HRV_C1d,HRV_C1a,HRV_SD1d,HRV_SD1a,HRV_C2d,HRV_C2a,HRV_SD2d,HRV_SD2a,HRV_Cd,HRV_Ca,HRV_SDNNd,HRV_SDNNa,HRV_ApEn,HRV_SampEn
0,78.438294,121.301195,1530.84373,211.665607,121.313923,0.138267,0.079238,1528.0,212.0118,0.138751,285.0,57.025835,79.101029,1672.0,14.831776,0.000509,0.002877,0.001854,0.000287,0.000002,6.451483,0.335333,0.051978,-8.154645,85.781898,286.769556,0.299132,77282.0381,3.343008,5.595049,3834.691482,0.47217,0.465047,0.719982,0.081103,49.964674,49.908816,50.025249,53.090367,0.582408,0.417592,65.465038,55.433425,0.450869,0.549131,192.556488,212.505946,0.461672,0.538328,143.811808,155.292694,1.440374,1.275916
